In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE133057"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE133057"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE133057.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE133057.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE133057.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic analysis of pretreated rectal cancer biopsies and association to the tumor regression score."
!Series_summary	"To determine a preditcive marker of treatment resistance for rectal cancer, we have employed a microarray gene profiling analysis on pretreated rectal biopsies and compared with their response to therapy as defined by the American Joint Commission on Cancer (AJCC) and the American College of Pathologists. "
!Series_overall_design	"Frozen rectal cancer biopsies utilized for the transcriptomic analysis were from 33 patients seen between 2006 and 2009 at Cleveland Clinic Main Campus in Cleveland, Ohio. After collection of biopsie and diagnosis, patients generally underwent surgery with curative intent approximately 8–12 weeks after completion of neoadjuvant chemoradiotherapy with 5-fluorouracil as radiation sensitizer and 50.4Gy in 25 fractions. The resected tumor is assessed by pathologists to determine AJCC score."
Sample Ch

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# as it's described as "transcriptomic analysis" and mentions microarray gene profiling analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait: AJCC score is available in row 1
trait_row = 1

# Age: Available in row 5
age_row = 5

# Gender: Available in row 2
gender_row = 2

# 2.2 Data Type Conversion

# For trait (AJCC score) - Ordinal/continuous data
def convert_trait(value):
    try:
        if ':' in value:
            # Extract the value after the colon
            value = value.split(':', 1)[1].strip()
        # Convert AJCC score to integer
        return int(value)
    except (ValueError, TypeError):
        return None

# For age - Continuous data
def convert_age(value):
    try:
        if ':' in value:
            # Extract the value after the colon
            value = value.split(':', 1)[1].strip()
        # Convert age to integer
        return int(value)
    except (ValueError, TypeError):
        return None

# For gender - Binary data (Female=0, Male=1)
def convert_gender(value):
    try:
        if ':' in value:
            # Extract the value after the colon
            value = value.split(':', 1)[1].strip()
        # Convert gender to binary (0=Female, 1=Male)
        if value.lower() == 'female':
            return 0
        elif value.lower() == 'male':
            return 1
        else:
            return None
    except (ValueError, TypeError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Perform initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since we don't have direct access to the clinical_data.csv file,
    # and because the format of the sample characteristics dictionary doesn't match
    # what's expected by geo_select_clinical_features, we need to:
    # 1. First check if the file exists through another path
    # 2. If not, reconstruct a properly formatted DataFrame
    
    clinical_data_path = f"{in_cohort_dir}/clinical_data.csv"
    
    try:
        # Try to load existing clinical data file if it exists
        clinical_data = pd.read_csv(clinical_data_path, index_col=0)
    except FileNotFoundError:
        # File doesn't exist, we need to manually construct the clinical data
        
        # Get the available sample characteristics
        sample_char_dict = {
            0: ['subject status: patient with rectal adenocarcinoma'], 
            1: ['ajcc score: 0', 'ajcc score: 1', 'ajcc score: 3', 'ajcc score: 2'], 
            2: ['gender: Female', 'gender: Male'], 
            3: ['overall survival (in days): 3182', 'overall survival (in days): 4584', 'overall survival (in days): 4452', 'overall survival (in days): 3789', 'overall survival (in days): 2960', 'overall survival (in days): 125', 'overall survival (in days): 4027', 'overall survival (in days): 1201', 'overall survival (in days): 403', 'overall survival (in days): 372', 'overall survival (in days): 3949', 'overall survival (in days): 3591', 'overall survival (in days): 647', 'overall survival (in days): 3964', 'overall survival (in days): 3837', 'overall survival (in days): 426', 'overall survival (in days): 2085', 'overall survival (in days): 858', 'overall survival (in days): 1147', 'overall survival (in days): 163', 'overall survival (in days): 3073', 'overall survival (in days): 3741', 'overall survival (in days): 3108', 'overall survival (in days): 3536', 'overall survival (in days): 2251', 'overall survival (in days): 2954', 'overall survival (in days): 2432', 'overall survival (in days): 1470', 'overall survival (in days): 969', 'overall survival (in days): 2000'], 
            4: ['dead (1)/alive(0): 0', 'dead (1)/alive(0): 1'], 
            5: ['age: 66', 'age: 65', 'age: 51', 'age: 72', 'age: 62', 'age: 50', 'age: 46', 'age: 59', 'age: 63', 'age: 44', 'age: 69', 'age: 41', 'age: 70', 'age: 54', 'age: 48', 'age: 75', 'age: 40', 'age: 47', 'age: 60', 'age: 43', 'age: 57', 'age: 52', 'age: 82']
        }
        
        # Since we can't directly use this data with geo_select_clinical_features,
        # we'll inform about the limitation and proceed without the clinical feature extraction
        print("Clinical data file not found and sample characteristics dictionary format is not compatible with geo_select_clinical_features.")
        print("Skipping clinical feature extraction step.")
        
        # We can still save the trait information to reflect we did the analysis
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
        
        # Without the proper clinical data format, we can't proceed with feature extraction
        # However, we've documented our analysis of the available variables
else:
    print("No trait data available for this cohort. Skipping clinical feature extraction.")


Clinical data file not found and sample characteristics dictionary format is not compatible with geo_select_clinical_features.
Skipping clinical feature extraction step.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['ILMN_1343289', 'ILMN_1343290', 'ILMN_1343291', 'ILMN_1343292',
       'ILMN_1343293', 'ILMN_1343294', 'ILMN_1343295', 'ILMN_1651199',
       'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651217', 'ILMN_1651221',
       'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651232', 'ILMN_1651234',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining gene identifiers
# The gene identifiers shown (ILMN_*) are Illumina probe IDs, not human gene symbols
# These are probe identifiers from Illumina microarray platforms and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1810835', 'ILMN_1758197'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_10478', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_175835', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'SPRR3', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_005416.1', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_005416.1', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 6707.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 4885606.0, 89033487.0], 'Accession': ['XM_933824.1', 'AK024680', 'XM_938851.1', 'NM_005416.1', 'XM_936379.1'], 'Symbol': ['LOC23117', nan, 'FCGR2B', 'S

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene identifier and gene symbol
probe_col = 'ID'  # This is the gene identifier column from the annotation
gene_col = 'Symbol'  # This is the gene symbol column from the annotation

# 2. Extract gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the converted gene expression data
print("First few rows of gene expression data after mapping:")
print(gene_data.head())
print(f"Shape of gene expression data: {gene_data.shape}")


First few rows of gene expression data after mapping:
          GSM3899156  GSM3899157   GSM3899158   GSM3899159  GSM3899160  \
Gene                                                                     
A1BG       -2.079415    2.469313     3.714346    -1.130043    1.139405   
A2BP1      -7.374455   -5.796633    -4.384497    14.895651  -11.592012   
A2M      3102.807000  822.027000  1090.359000  3902.472000  807.357700   
A2ML1       5.905322   18.653390     7.037081    15.808300   11.359380   
A3GALT2    62.152650   23.638210    27.884070    39.603730   33.517560   

          GSM3899161   GSM3899162   GSM3899163   GSM3899164   GSM3899165  ...  \
Gene                                                                      ...   
A1BG        8.019033    -0.400172     0.842883     1.740345     2.132458  ...   
A2BP1       2.240619     0.723286    -3.208117    33.192579     1.097275  ...   
A2M      1564.063000  1489.232000  4251.913000  5816.318000  3845.279000  ...   
A2ML1      19.115360  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Let's extract clinical data from the matrix file again to get the correct format
# First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background info and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the correct row indices and conversion functions
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # AJCC score is in row 1
    convert_trait=convert_trait,  # Use the previously defined function
    age_row=5,
    convert_age=convert_age,
    gender_row=2,
    convert_gender=convert_gender
)

# Save clinical features again with proper format
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features shape: {clinical_features.shape}")
print(f"Clinical features columns: {clinical_features.columns}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients with AJCC scores, focusing on response to chemoradiotherapy."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Clinical features shape: (3, 33)
Clinical features columns: Index(['GSM3899156', 'GSM3899157', 'GSM3899158', 'GSM3899159', 'GSM3899160',
       'GSM3899161', 'GSM3899162', 'GSM3899163', 'GSM3899164', 'GSM3899165',
       'GSM3899166', 'GSM3899167', 'GSM3899168', 'GSM3899169', 'GSM3899170',
       'GSM3899171', 'GSM3899172', 'GSM3899173', 'GSM3899174', 'GSM3899175',
       'GSM3899176', 'GSM3899177', 'GSM3899178', 'GSM3899179', 'GSM3899180',
       'GSM3899181', 'GSM3899182', 'GSM3899183', 'GSM3899184', 'GSM3899185',
       'GSM3899186', 'GSM3899187', 'GSM3899188'],
      dtype='object')
Normalized gene data shape: (17736, 33)
First few normalized gene symbols: ['A1BG', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSP1']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE133057.csv
Linked data shape: (33, 17739)
            Rectal_Cancer   Age  Gender      A1BG        A2M      A2ML1  \
GSM3899156            0.0  66.0     0.0 -2.079415  3102.8070   5.905322   
GSM3899157            1.0  65.0     0.0  2.469313   822.0270  18.653390   
GSM3899158            0.0  51.0     0.0  3.714346  1090.3590   7.037081   
GSM3899159            1.0  72.0     1.0 -1.130043  3902.4720  15.808300   
GSM3899160            0.0  62.0     1.0  1.139405   807.3577  11.359380   

             A3GALT2    A4GALT      A4GNT       AAA1  ...      ZWILCH  \
GSM3899156  62.15265  67.30495   4.375281  21.173169  ...   87.248004   
GSM3899157  23.63821  94.85796  23.498130  26.904985  ...   54.572260   
GSM3899158  27.88407  46.07135   3.783252  25.711841  ...  114.227830   
GSM3899159  39.60373  65.85915   7.583620  35.614006  ...   50.675996   
GSM3899160  33.51756  50.62852   6.398035  11.417144  ...   7

Shape after handling missing values: (33, 17739)
Quartiles for 'Rectal_Cancer':
  25%: 1.0
  50% (Median): 2.0
  75%: 2.0
Min: 0.0
Max: 3.0
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 54.0
  75%: 65.0
Min: 40.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 10 occurrences. This represents 30.30% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE133057.csv
